In [1]:
import os
import pickle

In [ ]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# Charger les données

data_path = "../data/processed"
file_name = 'dataset_final.pkl'
with open(os.path.join(data_path, file_name), 'rb') as f:
    df = pickle.load(f)

In [6]:
# Charger un seul embedding

embeddings_path = "../data/embeddings"
# file_name = 'emb_sbert_multi.pkl'

# with open(os.path.join(embeddings_path, file_name), 'rb') as f:
#     embedding = pickle.load(f)

In [4]:
# Charger tous les embeddings

# embeddings_path = "../data/embeddings"
# embeddings = {}

# for file_name in os.listdir(embeddings_path):
#     with open(os.path.join(embeddings_path, file_name), 'rb') as f:
#         emb_name = file_name.split('.')[0]
#         embeddings[emb_name] = pickle.load(f)

### BERTopic

In [4]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords

/mnt/c/Users/charb/Documents/ALTERNANCE/Datascientest/Projet/TrustPilot/DS/trustpilot/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/laurine/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
sentences = df["clean_comment"].tolist()

file_name_mutli = 'emb_sbert_multi.pkl'
with open(os.path.join(embeddings_path, file_name_mutli), 'rb') as f:
    embedding_multi = pickle.load(f)

file_name_fr = 'emb_sbert_fr.pkl'
with open(os.path.join(embeddings_path, file_name_fr), 'rb') as f:
    embedding_fr = pickle.load(f)

In [48]:
models = {}

for embedding, name in zip([embedding_multi, embedding_fr], ["multilingue", "français"]):

    vectorizer_model = CountVectorizer(
        stop_words=stopwords.words("french"),
        ngram_range=(1, 2),
        min_df=5
    )
    
    topic_model = BERTopic(
        language="french",
        vectorizer_model=vectorizer_model,
        verbose=True,
        nr_topics="auto"
    )
    
    topics, probs = topic_model.fit_transform(sentences, embedding_multi)

    models[name] = (topic_model)

2025-10-31 17:03:30,936 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-10-31 17:03:33,694 - BERTopic - Dimensionality - Completed ✓
2025-10-31 17:03:33,695 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-10-31 17:03:34,076 - BERTopic - Cluster - Completed ✓
2025-10-31 17:03:34,078 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-10-31 17:03:34,851 - BERTopic - Representation - Completed ✓
2025-10-31 17:03:34,852 - BERTopic - Topic reduction - Reducing number of topics
2025-10-31 17:03:34,893 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-10-31 17:03:35,677 - BERTopic - Representation - Completed ✓
2025-10-31 17:03:35,681 - BERTopic - Topic reduction - Reduced number of topics from 166 to 22
2025-10-31 17:03:35,770 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-10-31 17:03:37,854 - BERTopic - Dimensionality - Completed ✓
20

In [49]:
topic_model = models["français"]

In [50]:
# topics trouvés
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,6249,-1_commande_plus_livraison_colis,"[commande, plus, livraison, colis, service, si...",[je suis cliente depuis de nombreuses années m...
1,0,7855,0_très_commande_livraison_plus,"[très, commande, livraison, plus, qualité, bie...",[commande d'électroménager tekwood le 26 mars ...
2,1,132,1_taille_petit_trop_grand,"[taille, petit, trop, grand, trop grand, trop ...","[taille un peu petit ., déçue par ma commande ..."
3,2,127,2_boucles_téléphone_portable_plus,"[boucles, téléphone, portable, plus, mois, car...",[j ai acheté un portable reconditionné chez sh...
4,3,90,3_montre_site_fonctionne_acheté,"[montre, site, fonctionne, acheté, plus, fait,...",[je suis cliente de showroom . j'ai commandé u...
5,4,82,4_couleur_photo_déçue_jaune,"[couleur, photo, déçue, jaune, articles, tout,...","[la couleur ne me plait pas trop, un peu decu ..."
6,5,79,5_sac_beau_très beau_très,"[sac, beau, très beau, très, joli, main, bonne...","[très beau sac correspond à mon attente, achat..."
7,6,60,6_showroom_satisfaite_showroomprive_merci,"[showroom, satisfaite, showroomprive, merci, r...","[toujours satisfaite avec showroom, j'achète s..."
8,7,58,7_produits_vide_bon_très,"[produits, vide, bon, très, produit, comme, ca...","[j'ai commandé 5 produits qui , pour moi , son..."
9,8,57,8_téléphone_jours_sav_appareil,"[téléphone, jours, sav, appareil, achat, garan...",[j ai fait un achat et quel achat ...... un ip...


In [51]:
# Mots-clés associés à un topic
topic_model.get_topic(0)

[('très', np.float64(0.0253478604869382)),
 ('commande', np.float64(0.02415428401740931)),
 ('livraison', np.float64(0.022455089324755775)),
 ('plus', np.float64(0.01981827838253375)),
 ('qualité', np.float64(0.01836062707986705)),
 ('bien', np.float64(0.01783792594186758)),
 ('site', np.float64(0.01708127550574485)),
 ('colis', np.float64(0.016722186033247648)),
 ('article', np.float64(0.016507817450982565)),
 ('articles', np.float64(0.015977086160480575))]

In [52]:
topic_model.visualize_topics()

In [53]:
# Répartition des topics
topic_model.visualize_barchart()

In [54]:
# Hiérarchie des topics
topic_model.visualize_hierarchy()